<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#機械学習における前処理" data-toc-modified-id="機械学習における前処理-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>機械学習における前処理</a></span></li><li><span><a href="#階層カラムを持つデータフレームの操作方法" data-toc-modified-id="階層カラムを持つデータフレームの操作方法-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>階層カラムを持つデータフレームの操作方法</a></span></li><li><span><a href="#Permutaion-Importance" data-toc-modified-id="Permutaion-Importance-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Permutaion Importance</a></span><ul class="toc-item"><li><span><a href="#Eli5-パッケージを使う" data-toc-modified-id="Eli5-パッケージを使う-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Eli5 パッケージを使う</a></span></li></ul></li><li><span><a href="#良い教材サイト" data-toc-modified-id="良い教材サイト-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>良い教材サイト</a></span></li><li><span><a href="#３群以上における比率の検定" data-toc-modified-id="３群以上における比率の検定-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>３群以上における比率の検定</a></span><ul class="toc-item"><li><span><a href="#フィッシャーの正確確率検定" data-toc-modified-id="フィッシャーの正確確率検定-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>フィッシャーの正確確率検定</a></span></li><li><span><a href="#ANOVA" data-toc-modified-id="ANOVA-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>ANOVA</a></span><ul class="toc-item"><li><span><a href="#ANOVAの後の多重比較" data-toc-modified-id="ANOVAの後の多重比較-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>ANOVAの後の多重比較</a></span></li></ul></li></ul></li><li><span><a href="#Jupyter-Tips" data-toc-modified-id="Jupyter-Tips-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Jupyter Tips</a></span><ul class="toc-item"><li><span><a href="#自作モジュール変更や新規インストールしたモジュールを手動反映する" data-toc-modified-id="自作モジュール変更や新規インストールしたモジュールを手動反映する-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>自作モジュール変更や新規インストールしたモジュールを手動反映する</a></span></li></ul></li></ul></div>

# 概要
学習メモである。2019-02-04以降の内容である。


# 機械学習における前処理

おさらいであるが、良いエントリを見つけた。

Machine Learning(ML) — Data Preprocessing – Data Driven Investor – Medium
https://medium.com/datadriveninvestor/machine-learning-ml-data-preprocessing-5b346766fc48


機械学習における前処理は、主には次の５つに分けられる。 
* データクリーニング
    1. 欠損値を補完する
    2. ノイズを含むデータを平滑化する
    3. 異常値を識別する、または除去する
    4. その他不整合を解決する
* データ統合
    複数のデータベース、テーブル、ファイルを利用して、データを結合しする。
* データ変換
    正規化や集約処理をする
* データ削減
    データ量を減らしつつも、同じか類似した分析結果を得られるようにする。
* データ離散化
    数値をone-hot形式に変換するなど

> Data cleaning: fill in missing values, smooth noisy data, identify or remove outliers, and resolve inconsistencies.  
Data integration: using multiple databases, data cubes, or files.  
Data transformation: normalization and aggregation.  
Data reduction: reducing the volume but producing the same or similar analytical results.  
Data discretization: part of data reduction, replacing numerical attributes with nominal ones.  


# 階層カラムを持つデータフレームの操作方法
カラムが階層構造を持つ場合、タプルを用いて各階層のカラムを指定すれば良い。ワイルドカード指定をしたい場合は、カラム指定をスキップすれば良い。  

```python
# カラムは３階層構造を持っている。第一階層では "create_time" と "receive_time" を指定し、
# 第二階層では "LMAX_LD" を指定。
df.loc[df.index[0:100],(("create_time","receive_time"),"LMAX_LD")]
```

# Permutaion Importance
構築したモデルにおいて、どの特徴量が重要なのかを、各特徴量の並び順をランダムにシャッフルしながら求めたもの。  
モデルを構築しなおすのではなく、値を入れ替えたもので予測させるので、計算負荷に考慮した手法である。  

[Python: 特徴量の重要度を Permutation Importance で計測する - CUBE SUGAR CONTAINER](https://blog.amedama.jp/entry/permutation-importance)  
おなじみのiris dataset をランダムフォレストで学習させ、構築したモデルにおける各特徴量の影響度を Permutation Importance として計算。  
ランダムに並べ替えると性能悪化が大きい特徴量は ```Petal length```, ```Petal width``` である。  
性能悪化のばらつきの幅も大きい。  
一方、影響が少ないのは```Sepal length```, ```Sepal width``` で、性能悪化はほぼゼロ。ばらつきも小さい。   
![Permutation Importance](https://cdn-ak.f.st-hatena.com/images/fotolife/m/momijiame/20181110/20181110230620.png)


Permutation Importance の話題からそれるが、以下のように yield を使うと、イテレーション処理で呼び出すジェネレータを作ることが出来る。  
```python
def permuted(df):
    """特定のカラムをシャッフルしたデータフレームを返す"""
    for column_name in df.columns:
        permuted_df = df.copy()
        permuted_df[column_name] = np.random.permutation(permuted_df[column_name])
        yield column_name, permuted_df
        
# このようなイテレーション処理に使える。
for column_name, permuted_X_test in permuted(X_test):
    # 処理
```

## Eli5 パッケージを使う

Permutation Importance を計算するための```Eli5```というパッケージがある。  
以下のエントリで使い方を紹介している。  
[機械学習モデルにおける特徴量の重要度評価Permutation Importanceとeli5について 実装編 | 分析トレイン](https://bunseki-train.com/permutaion_importance_and_eli5_2/)

こちらも、お馴染みのワインデータセットをランダムフォレストで学習し、Eli5パッケージを使って Permutation Importance を求めている。  
(このような表形式でレンダリングする機能も提供されている。)
![Permutation Importance @ Wine](https://bunseki-train.com/wp/wp-content/uploads/2018/09/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-09-22-00.27.24-300x200.png)

そして、影響度上位２つの特徴量を散布図にすると、以下のような図が得られた。  
![Scatter Plot@Wine](https://bunseki-train.com/wp/wp-content/uploads/2018/09/output_4_0.png)

他にも、個々のサンプルにおける特徴量の重要度を計算するAPIも提供されている。  
```python
# コード例
eli5.show_prediction(rf_clf, doc=val_X.iloc[0], feature_names= val_X.columns.tolist(), show_feature_values=True)
```
![Permutation Importance@each sample](https://bunseki-train.com/wp/wp-content/uploads/2018/09/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-09-22-18.57.46-1024x232.png)

# 良い教材サイト

以下のサイトのRSSは、定点観測しておくべきだろう。
https://towardsdatascience.com/

# ３群以上における比率の検定

## フィッシャーの正確確率検定
2群の比較にしか対応しておらず。

In [1]:
import scipy.stats as stats
oddsratio, pvalue = stats.fisher_exact([[8, 2], [1, 5]])
pvalue

0.034965034965034947

In [2]:
oddsratio, pvalue = stats.fisher_exact([[8, 2,6], [1, 5,3]])


ValueError: The input `table` must be of shape (2, 2).

## ANOVA
ユーザーを3つのグループにわけ、それぞれのある広告に対するクリック率を分散分析にかける。  
以下の実験では、p値が 0.09426 > 0.05 であり、いずれの集団間のクリック率に差が無い模様。

In [5]:
# ３つのグループを用意する
group_1 = [0] * 450 + [1] * 30   # 480人中30人がクリック
group_2 = [0] * 1450 + [1] * 70   # 1520人中70人がクリック
group_3 = [0] * 250 + [1] * 20   # 270人中20人がクリック

print("group_1 : クリック率 {}".format(sum(group_1)/len(group_1)))
print("group_2 : クリック率 {}".format(sum(group_2)/len(group_2)))
print("group_3 : クリック率 {}".format(sum(group_3)/len(group_3)))

result = stats.f_oneway(group_1, group_2, group_3)
print(result)

group_1 : クリック率 0.0625
group_2 : クリック率 0.046052631578947366
group_3 : クリック率 0.07407407407407407
F_onewayResult(statistic=2.3640770772502058, pvalue=0.094267839828490102)


しかし、この分布が正規分布に従っているかどうか不明の場合、ノンパラメトリックANOVAを実施する必要がある。 
そこで使うのが クラスカル＝ウォリス検定である。  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html  

結果は、p=0.0943 > 0.05 であり、３群のクリック率に有意差は見られないということになる。

In [6]:
 stats.kruskal(group_1, group_2, group_3)

KruskalResult(statistic=4.7224760396281864, pvalue=0.094303401417893409)

グループ１のクリック人数が増えると、p値が大きく改善した。  
p = ```4.4366171784343485e-11``` < 0.05 

In [7]:
# ３つのグループを用意する
group_1 = [0] * 450 + [1] * 70   # 520人中70人がクリック
group_2 = [0] * 1450 + [1] * 70   # 1520人中70人がクリック
group_3 = [0] * 250 + [1] * 20   # 270人中20人がクリック

print("group_1 : クリック率 {}".format(sum(group_1)/len(group_1)))
print("group_2 : クリック率 {}".format(sum(group_2)/len(group_2)))
print("group_3 : クリック率 {}".format(sum(group_3)/len(group_3)))

result = stats.f_oneway(group_1, group_2, group_3)
print(result)

group_1 : クリック率 0.1346153846153846
group_2 : クリック率 0.046052631578947366
group_3 : クリック率 0.07407407407407407
F_onewayResult(statistic=24.086576429269371, pvalue=4.4366171784343485e-11)


In [13]:
# ３つのグループを用意する
group_1 = [0] * 450 + [1] * 34   
group_2 = [0] * 1450 + [1] * 70   # 1520人中70人がクリック
group_3 = [0] * 250 + [1] * 20   # 270人中20人がクリック

print("group_1 : クリック率 {}".format(sum(group_1)/len(group_1)))
print("group_2 : クリック率 {}".format(sum(group_2)/len(group_2)))
print("group_3 : クリック率 {}".format(sum(group_3)/len(group_3)))

result = stats.f_oneway(group_1, group_2, group_3)
print(result)

group_1 : クリック率 0.07024793388429752
group_2 : クリック率 0.046052631578947366
group_3 : クリック率 0.07407407407407407
F_onewayResult(statistic=3.2241185449912395, pvalue=0.039973044542158963)


In [14]:
result.pvalue

0.039973044542158963

### ANOVAの後の多重比較
ANOVAを行い、p値が 0.05 以下であれば、比較群の平均値はいずれも同じというわけではない可能性が高いので、
どの群とどの群の平均値が異なるのか、多重比較をするのだが、比較する値が違うので理論的には誤りであり、
慣例的に行われているに過ぎないという指摘があった。  
> ・分散分析の後、どの群間に差があったかを知る検定（多重比較法）  
※分散分析→多重比較 という手順は理論的に誤り；扱う統計量が異なるため  
　⇒慣習的に行われているのが現状  
　⇒様々な方法があるが、明らかに誤りである方法以外のものを選択すれば問題ないはず  
 http://chishiegg.com/archives/46945828.html
 


# Jupyter Tips
参考: https://qiita.com/ishizakiiii/items/b98bbf8997f039f40058

## 自作モジュール変更や新規インストールしたモジュールを手動反映する
```%reload_ext autoreload``` を実行すれば、変更したモジュールの内容をすぐにカーネルに反映できる。  
